In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist

# Datasets load

In [2]:
def load_mnist():
    import os
    path = os.path.join(os.getcwd(), 'mnist.npz')
    
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data(path)
    
    # [batch_size, height, width, color_channel]
    train_data = np.expand_dims(train_data, axis=-1)
    test_data = np.expand_dims(test_data, axis=-1)
    
    # data normalization
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0
    
    # One hot encoding
    train_labels = to_categorical(train_labels, 10)
    test_labels = to_categorical(test_labels, 10)
    
    return train_data, train_labels, test_data, test_labels

# Create model (activation function = relu)
layer => batch normalization => activation => drop out

In [3]:
class CreateModel(tf.keras.Model):
    def __init__(self, label_dims):
        super(CreateModel, self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        
        self.dense1 = tf.keras.layers.Dense(units=256, use_bias=True, kernel_initializer='he_uniform')
        self.batch_norm1 = tf.keras.layers.BatchNormalization()
        self.relu1 = tf.keras.layers.Activation(tf.keras.activations.relu)
        self.drop1 = tf.keras.layers.Dropout(rate=0.7)
        
        self.dense2 = tf.keras.layers.Dense(units=512, use_bias=True, kernel_initializer='he_uniform')
        self.batch_norm2 = tf.keras.layers.BatchNormalization()
        self.relu2 = tf.keras.layers.Activation(tf.keras.activations.relu)
        self.drop2 = tf.keras.layers.Dropout(rate=0.7)
        
        self.dense3 = tf.keras.layers.Dense(units=label_dims, use_bias=True, kernel_initializer='he_uniform')
        self.batch_norm3 = tf.keras.layers.BatchNormalization()
    
    def call(self, inputs, training=False):
        net = self.flatten(inputs)
        
        net = self.dense1(net)
        net = self.relu1(net)
        net = self.batch_norm1(net)
        
        net = self.dense2(net)
        net = self.relu2(net)
        net = self.batch_norm2(net)
        
        net = self.dense3(net)
        net = self.batch_norm3(net)
        
        return net

# Loss 정의하기

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, from_logits=True))
    return loss
    
def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32)) * 100
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

# 데이터 전처리

In [5]:
train_x, train_y, test_x, test_y = load_mnist()

learning_rate = 0.001
batch_size = 128
train_dataset_num = len(train_x) // batch_size

train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x), drop_remainder=True)

11493376/11490434 [==============================] - 9s 1us/step


# 학습 하기

In [6]:
label_dim = 10

model = CreateModel(label_dim)
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)

epoch = 1
for i in range(epoch):
    for idx, (images, labels) in enumerate(train_dataset):
        grads = grad(model, images, labels)
        optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))
        
        loss = loss_fn(model, images, labels)
        accuracy = accuracy_fn(model, images, labels)
        
        print('Epoch: {:5}, Process: {:5}/{}, Loss: {:5.5f}, Accuracy: {:5.1f}%'.format(i, idx, train_dataset_num, loss, accuracy))

Epoch:     0, Process:     0/468, Loss: 1.47165, Accuracy:  37.5%
Epoch:     0, Process:     1/468, Loss: 1.24856, Accuracy:  47.7%
Epoch:     0, Process:     2/468, Loss: 1.19742, Accuracy:  53.9%
Epoch:     0, Process:     3/468, Loss: 1.13269, Accuracy:  62.5%
Epoch:     0, Process:     4/468, Loss: 1.08959, Accuracy:  65.6%
Epoch:     0, Process:     5/468, Loss: 1.02629, Accuracy:  68.8%
Epoch:     0, Process:     6/468, Loss: 1.00367, Accuracy:  60.9%
Epoch:     0, Process:     7/468, Loss: 0.99949, Accuracy:  66.4%
Epoch:     0, Process:     8/468, Loss: 0.86569, Accuracy:  71.1%
Epoch:     0, Process:     9/468, Loss: 0.85141, Accuracy:  74.2%
Epoch:     0, Process:    10/468, Loss: 0.90543, Accuracy:  67.2%
Epoch:     0, Process:    11/468, Loss: 0.95452, Accuracy:  64.1%
Epoch:     0, Process:    12/468, Loss: 0.85247, Accuracy:  70.3%
Epoch:     0, Process:    13/468, Loss: 0.80103, Accuracy:  73.4%
Epoch:     0, Process:    14/468, Loss: 0.83472, Accuracy:  71.1%
Epoch:    

Epoch:     0, Process:    17/468, Loss: 0.80085, Accuracy:  71.1%
Epoch:     0, Process:    18/468, Loss: 0.80695, Accuracy:  73.4%
Epoch:     0, Process:    19/468, Loss: 0.80708, Accuracy:  71.1%
Epoch:     0, Process:    20/468, Loss: 0.74220, Accuracy:  71.9%
Epoch:     0, Process:    21/468, Loss: 0.73057, Accuracy:  74.2%
Epoch:     0, Process:    22/468, Loss: 0.77545, Accuracy:  79.7%
Epoch:     0, Process:    23/468, Loss: 0.75155, Accuracy:  77.3%
Epoch:     0, Process:    24/468, Loss: 0.74461, Accuracy:  78.1%
Epoch:     0, Process:    25/468, Loss: 0.74635, Accuracy:  82.0%
Epoch:     0, Process:    26/468, Loss: 0.64130, Accuracy:  85.9%
Epoch:     0, Process:    27/468, Loss: 0.69795, Accuracy:  80.5%
Epoch:     0, Process:    28/468, Loss: 0.70388, Accuracy:  82.0%
Epoch:     0, Process:    29/468, Loss: 0.62046, Accuracy:  88.3%
Epoch:     0, Process:    30/468, Loss: 0.73514, Accuracy:  82.0%
Epoch:     0, Process:    31/468, Loss: 0.70017, Accuracy:  84.4%
Epoch:    

Epoch:     0, Process:    35/468, Loss: 0.67663, Accuracy:  88.3%
Epoch:     0, Process:    36/468, Loss: 0.60756, Accuracy:  87.5%
Epoch:     0, Process:    37/468, Loss: 0.64584, Accuracy:  89.8%
Epoch:     0, Process:    38/468, Loss: 0.76010, Accuracy:  85.2%
Epoch:     0, Process:    39/468, Loss: 0.60583, Accuracy:  88.3%
Epoch:     0, Process:    40/468, Loss: 0.63094, Accuracy:  86.7%
Epoch:     0, Process:    41/468, Loss: 0.71816, Accuracy:  83.6%
Epoch:     0, Process:    42/468, Loss: 0.62843, Accuracy:  87.5%
Epoch:     0, Process:    43/468, Loss: 0.55347, Accuracy:  91.4%
Epoch:     0, Process:    44/468, Loss: 0.62000, Accuracy:  90.6%
Epoch:     0, Process:    45/468, Loss: 0.60316, Accuracy:  90.6%
Epoch:     0, Process:    46/468, Loss: 0.54042, Accuracy:  90.6%
Epoch:     0, Process:    47/468, Loss: 0.67504, Accuracy:  89.8%
Epoch:     0, Process:    48/468, Loss: 0.57566, Accuracy:  89.1%
Epoch:     0, Process:    49/468, Loss: 0.57704, Accuracy:  89.1%
Epoch:    

Epoch:     0, Process:    53/468, Loss: 0.55438, Accuracy:  93.8%
Epoch:     0, Process:    54/468, Loss: 0.57795, Accuracy:  94.5%
Epoch:     0, Process:    55/468, Loss: 0.53392, Accuracy:  93.8%
Epoch:     0, Process:    56/468, Loss: 0.59039, Accuracy:  91.4%
Epoch:     0, Process:    57/468, Loss: 0.57618, Accuracy:  85.9%
Epoch:     0, Process:    58/468, Loss: 0.56414, Accuracy:  92.2%
Epoch:     0, Process:    59/468, Loss: 0.58006, Accuracy:  91.4%
Epoch:     0, Process:    60/468, Loss: 0.61037, Accuracy:  87.5%
Epoch:     0, Process:    61/468, Loss: 0.61959, Accuracy:  89.8%
Epoch:     0, Process:    62/468, Loss: 0.56371, Accuracy:  90.6%
Epoch:     0, Process:    63/468, Loss: 0.53635, Accuracy:  92.2%
Epoch:     0, Process:    64/468, Loss: 0.56267, Accuracy:  89.8%
Epoch:     0, Process:    65/468, Loss: 0.58372, Accuracy:  89.1%
Epoch:     0, Process:    66/468, Loss: 0.55686, Accuracy:  95.3%
Epoch:     0, Process:    67/468, Loss: 0.54912, Accuracy:  89.8%
Epoch:    

Epoch:     0, Process:    71/468, Loss: 0.49814, Accuracy:  94.5%
Epoch:     0, Process:    72/468, Loss: 0.58647, Accuracy:  91.4%
Epoch:     0, Process:    73/468, Loss: 0.57544, Accuracy:  89.1%
Epoch:     0, Process:    74/468, Loss: 0.50284, Accuracy:  90.6%
Epoch:     0, Process:    75/468, Loss: 0.51544, Accuracy:  92.2%
Epoch:     0, Process:    76/468, Loss: 0.55555, Accuracy:  90.6%
Epoch:     0, Process:    77/468, Loss: 0.49991, Accuracy:  93.0%
Epoch:     0, Process:    78/468, Loss: 0.57243, Accuracy:  90.6%
Epoch:     0, Process:    79/468, Loss: 0.47371, Accuracy:  92.2%
Epoch:     0, Process:    80/468, Loss: 0.49559, Accuracy:  93.0%
Epoch:     0, Process:    81/468, Loss: 0.53788, Accuracy:  93.0%
Epoch:     0, Process:    82/468, Loss: 0.50359, Accuracy:  92.2%
Epoch:     0, Process:    83/468, Loss: 0.52149, Accuracy:  92.2%
Epoch:     0, Process:    84/468, Loss: 0.48809, Accuracy:  95.3%
Epoch:     0, Process:    85/468, Loss: 0.48644, Accuracy:  93.0%
Epoch:    

...

Epoch:     0, Process:   395/468, Loss: 0.27609, Accuracy:  96.9%
Epoch:     0, Process:   396/468, Loss: 0.25815, Accuracy:  95.3%
Epoch:     0, Process:   397/468, Loss: 0.24824, Accuracy:  98.4%
Epoch:     0, Process:   398/468, Loss: 0.25453, Accuracy:  97.7%
Epoch:     0, Process:   399/468, Loss: 0.25810, Accuracy:  95.3%
Epoch:     0, Process:   400/468, Loss: 0.27712, Accuracy:  96.1%
Epoch:     0, Process:   401/468, Loss: 0.24228, Accuracy:  96.9%
Epoch:     0, Process:   402/468, Loss: 0.31369, Accuracy:  94.5%
Epoch:     0, Process:   403/468, Loss: 0.27981, Accuracy:  97.7%
Epoch:     0, Process:   404/468, Loss: 0.23501, Accuracy:  98.4%
Epoch:     0, Process:   405/468, Loss: 0.27972, Accuracy:  96.9%
Epoch:     0, Process:   406/468, Loss: 0.28837, Accuracy:  98.4%
Epoch:     0, Process:   407/468, Loss: 0.27405, Accuracy:  97.7%
Epoch:     0, Process:   408/468, Loss: 0.24377, Accuracy:  97.7%
Epoch:     0, Process:   409/468, Loss: 0.20786, Accuracy: 100.0%
Epoch:    

Epoch:     0, Process:   413/468, Loss: 0.27015, Accuracy:  96.1%
Epoch:     0, Process:   414/468, Loss: 0.25616, Accuracy:  96.9%
Epoch:     0, Process:   415/468, Loss: 0.26003, Accuracy:  96.9%
Epoch:     0, Process:   416/468, Loss: 0.27108, Accuracy:  97.7%
Epoch:     0, Process:   417/468, Loss: 0.23649, Accuracy:  98.4%
Epoch:     0, Process:   418/468, Loss: 0.24983, Accuracy:  96.1%
Epoch:     0, Process:   419/468, Loss: 0.18960, Accuracy:  98.4%
Epoch:     0, Process:   420/468, Loss: 0.29832, Accuracy:  95.3%
Epoch:     0, Process:   421/468, Loss: 0.29284, Accuracy:  96.9%
Epoch:     0, Process:   422/468, Loss: 0.26401, Accuracy:  99.2%
Epoch:     0, Process:   423/468, Loss: 0.28409, Accuracy:  98.4%
Epoch:     0, Process:   424/468, Loss: 0.24971, Accuracy:  96.9%
Epoch:     0, Process:   425/468, Loss: 0.18887, Accuracy: 100.0%
Epoch:     0, Process:   426/468, Loss: 0.26329, Accuracy:  97.7%
Epoch:     0, Process:   427/468, Loss: 0.26691, Accuracy:  95.3%
Epoch:    

Epoch:     0, Process:   431/468, Loss: 0.25914, Accuracy:  96.9%
Epoch:     0, Process:   432/468, Loss: 0.22694, Accuracy:  99.2%
Epoch:     0, Process:   433/468, Loss: 0.25233, Accuracy:  98.4%
Epoch:     0, Process:   434/468, Loss: 0.22810, Accuracy:  97.7%
Epoch:     0, Process:   435/468, Loss: 0.27077, Accuracy:  97.7%
Epoch:     0, Process:   436/468, Loss: 0.24357, Accuracy:  96.1%
Epoch:     0, Process:   437/468, Loss: 0.20100, Accuracy:  99.2%
Epoch:     0, Process:   438/468, Loss: 0.25698, Accuracy:  99.2%
Epoch:     0, Process:   439/468, Loss: 0.26820, Accuracy:  98.4%
Epoch:     0, Process:   440/468, Loss: 0.23908, Accuracy: 100.0%
Epoch:     0, Process:   441/468, Loss: 0.20658, Accuracy:  98.4%
Epoch:     0, Process:   442/468, Loss: 0.29385, Accuracy:  96.1%
Epoch:     0, Process:   443/468, Loss: 0.23699, Accuracy:  98.4%
Epoch:     0, Process:   444/468, Loss: 0.28690, Accuracy:  98.4%
Epoch:     0, Process:   445/468, Loss: 0.23981, Accuracy:  96.9%
Epoch:    

Epoch:     0, Process:   449/468, Loss: 0.23630, Accuracy:  96.9%
Epoch:     0, Process:   450/468, Loss: 0.25929, Accuracy:  96.9%
Epoch:     0, Process:   451/468, Loss: 0.24159, Accuracy:  99.2%
Epoch:     0, Process:   452/468, Loss: 0.24411, Accuracy:  98.4%
Epoch:     0, Process:   453/468, Loss: 0.23114, Accuracy:  97.7%
Epoch:     0, Process:   454/468, Loss: 0.23882, Accuracy:  97.7%
Epoch:     0, Process:   455/468, Loss: 0.27124, Accuracy:  96.9%
Epoch:     0, Process:   456/468, Loss: 0.25328, Accuracy:  98.4%
Epoch:     0, Process:   457/468, Loss: 0.26646, Accuracy:  95.3%
Epoch:     0, Process:   458/468, Loss: 0.21539, Accuracy:  99.2%
Epoch:     0, Process:   459/468, Loss: 0.22404, Accuracy:  97.7%
Epoch:     0, Process:   460/468, Loss: 0.24426, Accuracy:  97.7%
Epoch:     0, Process:   461/468, Loss: 0.24451, Accuracy:  99.2%
Epoch:     0, Process:   462/468, Loss: 0.20962, Accuracy:  99.2%
Epoch:     0, Process:   463/468, Loss: 0.23333, Accuracy:  97.7%
Epoch:    

Epoch:     0, Process:   467/468, Loss: 0.21776, Accuracy:  98.4%


# Model 성능 테스트

random normalization => 96.2%<br>
glorot normalization => 96.5%<br>
he uniform => 96.8%

In [7]:
for images, labels in test_dataset:
    accuracy = accuracy_fn(model, images, labels)
    print('Accuracy: {:5.1f}%'.format(accuracy))

Accuracy:  97.2%
